In [19]:
import pandas as pd
import numpy as np
from astropy.io import fits
import utils
import os

simulations_pre_path = './simulations/fits_files/i/'

In [20]:
# reading our test dataset parameters
modelized_systems = pd.read_csv('test_dataset.csv')
modelized_systems.head()

,Unnamed: 0,OBJID-g,H0-g,Om0-g,exposure_time-g,numPix-g,pixel_scale-g,psf_type-g,read_noise-g,ccd_gain-g,...,PLANE_1-REDSHIFT-g,PLANE_2-REDSHIFT-g,PLANE_1-REDSHIFT-r,PLANE_2-REDSHIFT-r,PLANE_1-REDSHIFT-i,PLANE_2-REDSHIFT-i,PLANE_1-REDSHIFT-z,PLANE_2-REDSHIFT-z,PLANE_1-REDSHIFT-Y,PLANE_2-REDSHIFT-Y
0,1501,1502,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.320029,2.135518,0.320029,2.135518,0.320029,2.135518,0.320029,2.135518,0.320029,2.135518
1,2586,2587,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.455823,3.769438,0.455823,3.769438,0.455823,3.769438,0.455823,3.769438,0.455823,3.769438
2,2653,2654,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.768999,4.477702,0.768999,4.477702,0.768999,4.477702,0.768999,4.477702,0.768999,4.477702
3,1055,1056,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.723560,1.718039,0.723560,1.718039,0.723560,1.718039,0.723560,1.718039,0.723560,1.718039
4,705,706,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.389194,3.653703,0.389194,3.653703,0.389194,3.653703,0.389194,3.653703,0.389194,3.653703


In [21]:
# setting our prepare path script (it will set a galfit configuration path, image and psf as well)
file = open("./set_path.sh", "w") 
for i in range(0, 50):
    objid = int(modelized_systems['OBJID-g'][i])
    image_i = fits.open(simulations_pre_path+str(objid-1)+'.fits')[0].data
    teste = utils.lens_light_test(image_i)
    has_lens_light, f, x, y = teste.test()
    # only for those who has lens light
    if has_lens_light:
        file.write('python3 prepare_path.py '+ str(int(objid))+'\n')
# close files        
file.close() 

FileNotFoundError: [Errno 2] No such file or directory: './simulations/fits_files/i/1073.fits'

In [ ]:
# setting our lens light subtraction pipeline scripts
galfit_file = open("./lens_light_subtraction/galfit_lens_light.sh", "w") 
autolens_file = open("./autolens_lens_light.sh", "w") 
imfit_file = open("./imfit_lens_light.sh", "w") 
# run over # systems
for i in range(0, 50):
    objid = int(modelized_systems['OBJID-g'][i])
    image_i = fits.open(simulations_pre_path+str(objid-1)+'.fits')[0].data
    teste = utils.lens_light_test(image_i)
    has_lens_light, f, x, y = teste.test()
    # only for those who has lens light
    if has_lens_light:
        galfit_file.write('/home/joao/projects/programs/galfit/galfit ./'+ str(int(objid))+'/galfit_ell.feedme \n')
        autolens_file.write('python3 AutoLens_lens_light.py '+ str(int(objid))+ '\n')
        imfit_file.write('python3 ImFit_lens_light.py '+ str(int(objid))+ '\n')
# close files
galfit_file.close()
autolens_file.close()
imfit_file.close()